In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
print('The dataset is extracted')
extraction_path=r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

image_path = os.path.join(extraction_path, r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset')  # Replace with actual image path
img = mpimg.imread(image_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
import os
normal = os.listdir(r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\normal')
print(normal)
print(len(normal))

In [ ]:
import os
parkinson = os.listdir(r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\parkinson')
print(parkinson)
print(len(parkinson))

### Creating labels for Normal and Parkinson
#### Parkinson-->1
#### Normal-->0

In [3]:
normal_label=[0]*610

parkinson_label=[1]*221

In [ ]:
print(normal_label)
print(parkinson_label)

In [ ]:
print(len(normal_label))
print(len(parkinson_label))

In [ ]:
labels = normal_label + parkinson_label
labels

In [ ]:
print(len(labels))

**Displaying the image**


In [ ]:
# displaying with normal image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

img = mpimg.imread(r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\normal\Mag_Images_025.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
# displaying parkinson disease image
img = mpimg.imread(r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\parkinson\DUAL_TSE_009.png')
imgplot = plt.imshow(img)
plt.show()

### Image Processing
   ##### 1.Resize image
   ##### 2.Convert the images to numpy arrays


In [15]:
from PIL import Image
import numpy as np
import os

normal_data = r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\normal'
parkinson_data = r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\parkinson'

normal_files = os.listdir(normal_data)
parkinson_files = os.listdir(parkinson_data)

data = []

# Process normal images
for img_file in normal_files:
    img_path = os.path.join(normal_data, img_file)
    image = Image.open(img_path)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)

# Process parkinson images
for img_file in parkinson_files:
    img_path = os.path.join(parkinson_data, img_file)
    image = Image.open(img_path)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)


In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
data[800]

In [ ]:
type(data[0])

In [ ]:
data[0].shape

In [21]:
# converting image list and label list to numpy arrays

x = np.array(data)
y = np.array(labels)

In [ ]:
x

In [ ]:
type(x)

In [ ]:
y

In [ ]:
print(x.shape)
print(y.shape)

#### Splitting data into train and test

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_train.shape

In [ ]:
Y_test.shape

In [35]:
# scaling the data

X_train_scaled = X_train/255

X_test_scaled = X_test/255

In [ ]:
X_train_scaled[0]

In [ ]:
X_test_scaled[0]

### Building CNN

In [38]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

In [ ]:
num_of_classes = 2

cnn = keras.Sequential()

cnn.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
cnn.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

cnn.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
cnn.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

cnn.add(keras.layers.Flatten())

cnn.add(keras.layers.Dense(128, activation='relu'))
cnn.add(keras.layers.Dropout(0.5))

cnn.add(keras.layers.Dense(64, activation='relu'))
cnn.add(keras.layers.Dropout(0.5))


cnn.add(keras.layers.Dense(num_of_classes, activation='sigmoid'))


In [ ]:
cnn.summary()

In [49]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = cnn.fit(X_train_scaled, Y_train, epochs=5, validation_data=(X_test_scaled, Y_test))

In [ ]:
# Function to plot the accuracy graph
def plot_accuracy(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plotting the accuracy graph
plot_accuracy(history)

In [ ]:
cnn.save('parkinson.h5')

### Model Evaluation

In [ ]:
accuracy = cnn.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)

### Predictive System

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models


input_image_path = r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinsons_dataset\parkinson\dReg_-_sDW_SSh_SENSE_003.png'

input_image = cv2.imread(input_image_path)


input_image_resized = cv2.resize(input_image, (128,128))

input_image_scaled = input_image_resized/255

input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])

input_prediction = cnn.predict(input_image_reshaped)

print(input_prediction)


input_pred_label = np.argmax(input_prediction)

print(input_pred_label)


if input_pred_label == 0:

  print('The person is Normal')

else:

  print('The person is having parkinson disease')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assuming you have already loaded and preprocessed your data
# and your model is trained and saved as 'parkinson_cnn_model.h5'
from tensorflow.keras.models import load_model
model = load_model(r'C:\Users\Nandu Gatla\Downloads\parkinsons_dataset project\parkinson.h5')

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Generate confusion matrix
cm = confusion_matrix(Y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

